In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt


In [2]:
import sys
sys.path.append('../../../Code')
import loadData 
import RunML
import FS
import metric

2025-03-28 02:27:25.951827: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-28 02:27:25.956589: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-28 02:27:25.968159: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743143245.986396 3103256 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743143245.992058 3103256 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-28 02:27:26.013681: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [3]:

ARF_16S_metadata = pd.read_csv('../data/ARF_16S_metadata.csv',index_col = 0)
ARF_16S_ctb_family = pd.read_csv('../data/ARF_16S_ctb_family.csv',index_col = 0)

FileNotFoundError: [Errno 2] No such file or directory: '../data/ARF_16S_ctb_family.csv'

In [ ]:
ARF_16S_metadata.head(3)

In [ ]:
ARF_16S_ctb_family

In [ ]:
featurs_family = ARF_16S_ctb_family.columns

In [ ]:
featurs_family

In [ ]:
target_variable = ARF_16S_metadata['Phase']
target_variable

### Select feartures using SelectMicro

In [ ]:
selectedresult=FS.SelectMicro_fun(ARF_16S_ctb_family,target_variable,p_cutoff = 0.05)
    
selectedresult.keys()

In [ ]:
data = selectedresult['relative_abundance_data']


selectedOTU_index= selectedresult['selected_indices']
selectedASV = selectedresult['selected_columnames']
X_FS = selectedresult['selected_data']


In [ ]:

print("The response variable and its counts are:")
print(target_variable.value_counts())

In [ ]:
print(f'The shape of the original dataset is ',np.shape(data))
print(f'The shape of the SelectMicro dataset is ',np.shape(X_FS))


In [ ]:
#from sklearn.preprocessing import LabelEncoder
#le = LabelEncoder()
#y = le.fit_transform(target_variable)

In [ ]:
custom_phaseorder = ['Initial', 'BLOOM', 'CLIMAX', 'DECLINE', 'RECOVERY']
custom_mapping = {category: i for i, category in enumerate(custom_phaseorder)}

In [ ]:
custom_mapping

In [ ]:
y = [custom_mapping[category] for category in target_variable]
y = np.array(y)
print(y)

print(target_variable)

In [ ]:
X_lasso_ft,selectedOTU_index_Lasso  = RunML.LassoFS_CV(data,y)

X_FS_lasso_ft,xlabel_FS_lasso_ft0  = RunML.LassoFS_CV(X_FS,y)
selectedOTU_index_FS_lasso = selectedOTU_index[xlabel_FS_lasso_ft0]

data_subset = {"AllFeatures":data,
               "SelectMicro": X_FS,
               "Lasso_finetune":X_lasso_ft,
               "FS_Lassofinetune":X_FS_lasso_ft
              }

In [ ]:
print(f'The shape of the original dataset is ',np.shape(data))
print(f'The shape of the SelectMicro dataset is ',np.shape(X_FS))
print(f'The shape of the Lasso_finetune selected dataset is ',np.shape(X_lasso_ft))
print(f'The shape of the FS_Lasso_finetune selected dataset is ',np.shape(X_FS_lasso_ft))


In [ ]:
for datatype, subset in data_subset.items():   
    RunML.plot_dim_reduction(subset, y,method='PCA', datalabel=datatype)

In [ ]:
for datatype, subset in data_subset.items():   
    RunML.plot_dim_reduction(subset, y,method='TSNE', datalabel=datatype)

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import auc, roc_auc_score, roc_curve, accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, RocCurveDisplay, confusion_matrix, ConfusionMatrixDisplay
import shap
def RF_model_SCV_multi(X, y, plot=False,columnnames=None,SMOTE=False,k=5):
     # Initialize the classifier
   
    clf = RandomForestClassifier(n_jobs=5, random_state=777)

    # Set up 5-fold cross-validation
    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=777)

    # Lists to store the results
    accuracies = []
    roc_aucs = []
    mcc_s = []
    f_scores = []
    y_true_all = []
    
    x_true_all = np.empty((0, X.shape[1]))  # Ensure correct number of columns
    shap_values_all = np.empty((0, X.shape[1]))   # Store SHAP values for each fold
    y_pred_all = []
    y_prob_all = []
    

    for train_index, test_index in kf.split(X, y):
        # Split the data
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Apply SMOTE if specified
        if SMOTE:
            X_train, y_train = RunML.perform_SMOTE(X_train, y_train)
     
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        y_prob = clf.predict_proba(X_test)## For multiclass, y_prob will have shape (n_samples, n_classes)

        
        # Evaluate the model
        accuracy = accuracy_score(y_test, y_pred)
        roc_auc = roc_auc_score(y_test, y_prob, multi_class='ovr', average='macro')
        #mcc = metric.mcc_score(y_test,y_pred)
        f_score = f1_score(y_test,y_pred, average='weighted')

        explainer = shap.TreeExplainer(clf)
        shap_values = explainer.shap_values(X_test)
        #print(X_test.shape,shap_values.shape)
        
        accuracies.append(accuracy)
        roc_aucs.append(roc_auc)
        #mcc_s.append(mcc)
        f_scores.append(f_score)
        #shap_values_list.append(shap_values[:,:,0])
        #shap_values_list.append(np.abs(pd.DataFrame(shap_values[:,:,0]).values).mean(0))
        y_true_all.extend(y_test)# Using extend to add multiple elements
        x_true_all = np.vstack((x_true_all, X_test))
        shap_values_all = np.vstack((shap_values_all,shap_values[:, :, 0]))# save the SHAP of [0] in the result
        y_pred_all.extend(y_pred)
        y_prob_all.extend(y_prob.argmax(axis=1))  # For multiclass, take the class with the highest probability


    result = {'mean_accuracy': np.mean(accuracies),
              'std_accuracy':np.std(accuracies),
              'mean_auc': np.mean(roc_aucs),
              'std_auc':np.std(roc_aucs),
              #'mean_mcc':np.mean(mcc_s),
              'mean_f':np.mean(f_scores),
              'y_true': y_true_all,
              'y_pred':y_pred_all,
              'y_pred_prob': y_prob_all,
              'x_true':x_true_all,
              'SHAP':shap_values_all}
    if plot:
        x_true_df = pd.DataFrame(x_true_all, columns=columnnames)
        shap.summary_plot(shap_values_all, x_true_df)
    return result

In [ ]:
dict_cm = RF_model_SCV_multi(X_FS, y, plot=True,columnnames=selectedASV,SMOTE=True,k=5)

In [ ]:
dict_cm_full = RF_model_SCV_multi(data, y, plot=True,columnnames=featurs_family,SMOTE=True,k=5)


In [ ]:
dict_cm_lasso = RF_model_SCV_multi(X_lasso_ft, y, plot=True,columnnames=featurs_family[selectedOTU_index_Lasso],SMOTE=True,k=5)


In [ ]:
dict_cm_fs_lasso = RF_model_SCV_multi(X_FS_lasso_ft, y, plot=True,columnnames=featurs_family[selectedOTU_index_FS_lasso],SMOTE=True,k=5)
print(dict_cm_fs_lasso['mean_accuracy'],dict_cm_fs_lasso['mean_auc'],dict_cm_fs_lasso['mean_f'])

In [ ]:
print(f'The shape of the original dataset is ',np.shape(data))
print(f'The shape of the SelectMicro dataset is ',np.shape(X_FS))
print(f'The shape of the Lasso_finetune selected dataset is ',np.shape(X_lasso_ft))
print(f'The shape of the FS_Lasso_finetune selected dataset is ',np.shape(X_FS_lasso_ft))


In [ ]:
print(dict_cm_full['mean_accuracy'],dict_cm_full['mean_auc'],dict_cm_full['mean_f'])
print(dict_cm['mean_accuracy'],dict_cm['mean_auc'],dict_cm['mean_f'])
print(dict_cm_lasso['mean_accuracy'],dict_cm_lasso['mean_auc'],dict_cm_lasso['mean_f'])
print(dict_cm_fs_lasso['mean_accuracy'],dict_cm_fs_lasso['mean_auc'],dict_cm_fs_lasso['mean_f'])